In [1]:
import openai
import os

openai.api_type = "azure"
openai.api_base = "https://gpt3tests.openai.azure.com/"
openai.api_version = "2022-12-01"
openai.api_key = os.environ["OPENAI_API_KEY"]

engine = "Davinci003"
max_tokens = 1000

ModuleNotFoundError: No module named 'openai'

#### Use LLM class from `langchain` library

In [2]:
from langchain.llms import AzureOpenAI

openai_model = AzureOpenAI(deployment_name=engine)
openai_model.openai_api_key = openai.api_key
openai_model.openai_api_base = openai.api_base 
openai_model.max_tokens = max_tokens

#### Manual generation using OpenAI library

In [3]:
from typing import List

def openai_manual_generate(prompt: str, engine: str, max_tokens: int) -> List[str]:
    """
    Generate an answer using OpenAI
    """
    response = openai.Completion.create(
        engine=engine, prompt=prompt, max_tokens=max_tokens
    )
    
    return [choice["text"] for choice in response["choices"]]

## 1 - Prompt templates

### 1.1 - Test without chain of thought

In [4]:
prompt = "In what year was the film Departed with Leopnardo Dicaprio released? What is this year raised to the 0.43 power?"
openai_manual_generate(prompt, engine, max_tokens)

['\n\nThe film The Departed was released in 2006.\n\n2006 raised to the 0.43 power is approximately 2.356.']

In [5]:
pow(2006, 0.43)

26.30281917656938

### 1.2 - Test with chain of thought

In [6]:
prompt_2 = """Question: Can Barack Obama have a conversation with George Washington?

Give me a step by a step reasoning.

Answer: """
openai_manual_generate(prompt_2, engine, max_tokens)

['\nStep 1: Determine if this is possible in the physical world.\n\nNo, it is not possible for Barack Obama and George Washington to have a conversation in the physical world because George Washington is deceased. \n\nStep 2: Determine if this is possible in the spiritual world.\n\nIt is unclear whether it is possible for Barack Obama and George Washington to have a conversation in the spiritual world as this is an unanswered question.']

### 1.3 - Template with chain of thought

In [7]:
from langchain import PromptTemplate

template = """Question: {question}

Give me a step by a step reasoning.

Answer: """

prompt_template = PromptTemplate(template=template, input_variables=["question"])

In [9]:
prompt_3 = prompt_template.format(question="Can Albert Einstein have a conversation with George Washington?")
openai_manual_generate(prompt_3, engine, max_tokens)

['\nStep 1: Check if it is possible to bring Albert Einstein and George Washington together- No, it is not possible as Albert Einstein is deceased and George Washington died over 200 years ago. \n\nStep 2: Consider if technology can make it possible – No, currently there is not a technology that can bring deceased people back to life or communicate with people from the past. \n\nStep 3: Consider if there is a way for them to communicate- No, as both Albert Einstein and George Washington do not exist in the present, it is not possible for them to have a conversation.']

## 2 - Chains

In [10]:
from langchain import LLMChain

llm_chain = LLMChain(prompt=prompt_template, llm=openai_model)

question = "Can Barack Obama have a conversation with George Washington?"

print(llm_chain.run(question))


Step 1: Determine if Barack Obama and George Washington are alive. 
Step 2: If they are both alive, they can communicate directly via a telephone call, video chat, or in-person conversation.
Step 3: If only one of them is alive, they can communicate through a medium such as a psychic or medium.
Step 4: If neither of them is alive, then they cannot have a direct conversation. They could, however, communicate through something like a Ouija board or other paranormal activity.


## 3 - Agents and tools

Agents involve an LLM making decisions about which Actions to take, taking that Action, seeing an Observation, and repeating that until done.


When used correctly agents can be extremely powerful. In order to load agents, you should understand the following concepts:

- **Tool**: A function that performs a specific duty. This can be things like: Google Search, Database lookup, Python REPL, other chains.
- **LLM**: The language model powering the agent.
- **Agent**: The agent to use.

Tools: https://python.langchain.com/en/latest/modules/agents/tools.html

Agent Types: https://python.langchain.com/en/latest/modules/agents/agents/agent_types.html

In [11]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent

tools = load_tools(["wikipedia", "llm-math"], llm=openai_model)
agent = initialize_agent(tools, openai_model, agent="zero-shot-react-description", verbose=True)

In [12]:
agent.run("In what year was the film Departed with Leopnardo Dicaprio released? What is this year raised to the 0.43 power?")



> Entering new AgentExecutor chain...
 This question requires a bit of research. 
Action: Wikipedia
Action Input: Departed film

/anaconda/envs/gpt-tests/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /anaconda/envs/gpt-tests/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: Page: The Departed
Summary: The Departed is a 2006 American epic crime thriller film directed by Martin Scorsese and written by William Monahan. It is both a remake of the 2002 Hong Kong film Infernal Affairs and also loosely based on the real-life Boston Winter Hill Gang; the character Colin Sullivan is based on the corrupt FBI agent John Connolly, while the character Frank Costello is based on Irish-American gangster and crime boss Whitey Bulger. The film stars Leonardo DiCaprio, Matt Damon, Jack Nicholson, and Mark Wahlberg, with Martin Sheen, Ray Winstone, Vera Farmiga, Alec Baldwin, Anthony Anderson and James Badge Dale in supporting roles.
The film takes place in Boston and the surrounding metro area, primarily in the city’s South End neighborhood. Irish Mob boss Frank Costello (Nicholson) plants Colin Sullivan (Damon) as a spy within the Massachusetts State Police; simultaneously, the police assign undercover state trooper Billy Costigan (DiCaprio) to infiltrate Co

'The film Departed with Leonardo Dicaprio was released in 2006 and this year raised to the 0.43 power is 26.30281917656938.'

## 4 - Memory

Memory is the concept of persisting state between calls of a chain/agent. 

LangChain provides a standard interface for memory, a collection of memory implementations, and examples of chains/agents that use memory.

In [13]:
from langchain import ConversationChain

conversation = ConversationChain(llm=openai_model, verbose=True)

In [14]:
conversation.predict(input="Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


" Hi there! It's great to meet you. My name is AI and I'm here to help. What can I do for you today?"

In [15]:
conversation.predict(input="Can we talk about AI?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's great to meet you. My name is AI and I'm here to help. What can I do for you today?
Human: Can we talk about AI?
AI:

> Finished chain.


' Absolutely! What would you like to know about AI?'

In [16]:
conversation.predict(input="I'm interested in Reinforcement Learning.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's great to meet you. My name is AI and I'm here to help. What can I do for you today?
Human: Can we talk about AI?
AI:  Absolutely! What would you like to know about AI?
Human: I'm interested in Reinforcement Learning.
AI:

> Finished chain.


" Sure! Reinforcement Learning is a type of Machine Learning technique where an AI learns to maximize some notion of reward by interacting with its environment. It is commonly used in robotics, game playing and other types of decision making tasks. Is there anything else you'd like to know about Reinforcement Learning?"

## 5 - Document loaders

Combining language models with your own text data is a powerful way to differentiate them. The first step in doing this is to load the data into “documents” - a fancy way of say some pieces of text. This module is aimed at making this easy.

https://python.langchain.com/en/latest/modules/indexes/document_loaders.html

In [17]:
from langchain.document_loaders import NotionDirectoryLoader

loader = NotionDirectoryLoader("Notion_DB")

docs = loader.load()

## 7 - Indexes

Indexes refer to ways to structure documents so that LLMs can best interact with them. This module contains utility functions for working with documents

- **Embeddings**: An embedding is a numerical representation of a piece of information, for example, text, documents, images, audio, etc.
- **Text Splitters**: When you want to deal with long pieces of text, it is necessary to split up that text into chunks.
- **Vectorstores**: Vector databases store and index vector embeddings from NLP models to understand the meaning and context of strings of text, sentences, and whole documents for more accurate and relevant search results.

In [28]:
import requests

# Download file
url = "https://raw.githubusercontent.com/hwchase17/langchain/master/docs/modules/state_of_the_union.txt"
res = requests.get(url)
with open("state_of_the_union.txt", "w") as f:
  f.write(res.text)
  
# Document Loader
from langchain.document_loaders import TextLoader
loader = TextLoader('./state_of_the_union.txt')
documents = loader.load()

# Text Splitter
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [27]:
# Embeddings
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

# text = "This is a test document."
# query_result = embeddings.embed_query(text)
# doc_result = embeddings.embed_documents([text])
# doc_result

In [29]:
# Vectorstore: https://python.langchain.com/en/latest/modules/indexes/vectorstores.html
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, embeddings)

query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)

In [31]:
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


In [32]:
db.save_local("faiss_index")
new_db = FAISS.load_local("faiss_index", embeddings)
docs = new_db.similarity_search(query)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
